In [1]:
import numpy
import pandas as pd
import numpy as np
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator
import networkx as nx
import pandas as pd
import sklearn.metrics as metrics 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
def features(data):
       
    data.loc[data.HEAP == 0.0, 'HEAP'] = 0
    data.loc[data.HEAP != 0.0, 'HEAP'] = 1
    
    data.loc[data.WIC == 0.0 ,'WIC'] = 0
    data.loc[data.WIC != 0.0 ,'WIC'] = 1
    
    data.loc[data.SchoolLunch == 0.0, 'SchoolLunch'] = 0
    data.loc[data.SchoolLunch != 0.0, 'SchoolLunch'] = 1

    data.loc[data.MOOP == 0.0, 'MOOP'] = 0
    data.loc[data.MOOP != 0.0, 'MOOP'] = 1

    data.loc[data.Childcare == 0.0, 'Childcare'] = 0
    data.loc[data.Childcare != 0.0, 'Childcare'] = 1

    data.loc[data.FoodStamps == 0.0, 'FoodStamps'] = 0
    data.loc[data.FoodStamps != 0.0, 'FoodStamps'] = 1

    data.loc[data.Commuting == 0.0, 'Commuting'] = 0
    data.loc[data.Commuting != 0.0, 'Commuting'] = 1

    data.loc[data.Housing == 0.0, 'Housing'] = 0
    data.loc[data.Housing != 0.0, 'Housing'] = 1
    
    data = data.loc[:, [u'PovunitType', u'WorkExpIndiv',
                  u'HEAP', u'CitizenStatus',
                  u'EducAttain',
                  u'WIC', u'HousingStatus',
                  u'WorkersEquiv', u'SchoolLunch',
                  u'MOOP', u'MV', u'Childcare', 
                  u'FoodStamps', u'Commuting',
                  u'Housing', 'Pov_risk']]
    
    data.HousingStatus.replace({1:0, 2:1,3:2, 4:3,5:4, 6:5,7:6, 8:7,9:8}, inplace=True)
    data.WorkersEquiv.replace({1:0, 2:1,3:2, 4:3,5:4}, inplace=True)
    data.EducAttain.replace({1:0, 2:1,3:2, 4:3}, inplace=True)
    
    return data

In [3]:
# thresholds = pd.Series(puma_05.CEO_Threshold.unique())

In [4]:
# thresholds.hist(bins=35)
# plt.vlines(x=thresholds.median(), ymin=0, ymax=18, color = 'red')
# plt.vlines(x=thresholds.mean(), ymin=0, ymax=18, color = 'yellow')

In [5]:
# puma_05[puma_05.CEO_Threshold == puma_05.CEO_Threshold.max()]

In [6]:
# thresholds.describe()

In [7]:
# thresholds.quantile(0.99)

In [8]:
# thresholds_of = pd.Series(puma_05.Official_Threshold.unique())

In [9]:
# thresholds_of.hist(bins=30)

# New check without aggregation

In [10]:
# puma_05.CEO_Income.describe()

In [11]:
# puma_05[puma_05.CEO_Income < 0]['PUMA'].hist()

In [12]:
# puma_05[puma_05.SERIALNO == 812]

In [13]:
# puma_05[puma_05.CEO_Income == 0]

In [14]:
# puma_05.CEO_Income.quantile(0.1)

In [15]:
# df[(df.CEO_Poverty == 1)&(df.CEO_Threshold < df.CEO_Income)][['CEO_Threshold', 'CEO_Income','CEO_Poverty', 'Pov_risk', 'Threshold1']]

In [16]:
# df[(df.Pov_risk == 1)&((df.CEO_Poverty == 1))][['CEO_Threshold', 'CEO_Income','CEO_Poverty', 'Pov_risk', 'Threshold1']]

In [17]:
# X = df[[ u'PovunitType', u'WorkExpIndiv', u'Povunit',
#         u'SchoolBreakfast', u'HEAP', u'CitizenStatus',
#         u'AgeCateg_2', u'AgeCateg_3', u'EducAttain',
#         u'WIC', u'HousingStatus',
#        u'WorkersEquiv', u'SchoolLunch', u'AgeCateg_1',
#        u'MOOP', u'MV', u'Childcare', 
#        u'NP', u'FoodStamps', u'Commuting',
#        u'Housing', 'Pov_risk']]

In [18]:
# X.columns

# Making continuous variables discrete

In [19]:
# print X.SchoolBreakfast.value_counts()
# X.SchoolBreakfast.hist()

In [20]:
# print (X.HEAP).value_counts()
# X.HEAP.hist(bins= 50)

In [21]:
# X.HEAP[X.HEAP == 0.0] = 0
# X.HEAP[X.HEAP != 0.0] = 1

In [22]:
# X.HEAP.value_counts()

In [23]:
# print X.WIC.value_counts()
# X.WIC.hist(bins=50)

In [24]:
# X.WIC[X.WIC == 0.0] = 0
# X.WIC[X.WIC != 0.0] = 1

In [25]:
# X.WIC.value_counts()

In [26]:
# print X.SchoolLunch.value_counts()
# X.SchoolLunch.hist(bins = 100)

In [27]:
# X.SchoolLunch[X.SchoolLunch == 0.0] = 0
# X.SchoolLunch[X.SchoolLunch != 0.0] = 1

In [28]:
# X.SchoolLunch.value_counts()

In [29]:
# print X.MOOP.describe()
# (X.MOOP).hist(bins = 100)

In [30]:
# X.MOOP[X.MOOP == 0.0] = 0
# X.MOOP[X.MOOP != 0.0] = 1

In [31]:
# X.MOOP.plot.box()

In [32]:
# print X.Childcare.value_counts()
# X.Childcare.hist(log = True, bins=100)

In [33]:
# X.Childcare[X.Childcare == 0.0] = 0
# X.Childcare[X.Childcare != 0.0] = 1

In [34]:
# print X.FoodStamps.value_counts()
# X.FoodStamps.hist(bins=100, log = True)

In [35]:
# np.sqrt(X.FoodStamps).hist(bins=100, log = True)

In [36]:
# X.FoodStamps[X.FoodStamps == 0.0] = 0
# X.FoodStamps[X.FoodStamps != 0.0] = 1

In [37]:
# X.FoodStamps.value_counts()

In [38]:
# print X.Commuting.value_counts()
# X.Commuting.hist(log = True, bins=100)

In [39]:
# X[(((X.Commuting - X.Commuting.mean()) / X.Commuting.std()) < 0) &
#   (((X.Commuting - X.Commuting.mean()) / X.Commuting.std()) >= -1)].Pov_risk.value_counts()

In [40]:
# np.sqrt(X.Commuting).hist(bins= 100)

In [41]:
# X[(np.sqrt(X.Commuting) < 10)].Pov_risk.value_counts()

In [42]:
# X.Commuting[X.Commuting == 0.0] = 0
# X.Commuting[X.Commuting != 0.0] = 1

In [43]:
# X.Housing.unique()
# X.Housing.hist()

In [44]:
# X.Housing[X.Housing == 0.0] = 0
# X.Housing[X.Housing != 0.0] = 1

In [45]:
# G = nx.DiGraph()
# G.add_edges_from(best_model.edges())
# nx.draw(G, with_labels=True, arrows=True, node_size=700)

# Bayes Net Updated Columns

In [46]:
cols = ['WorkersEquiv', 'SchoolLunch', 'FoodStamps', 'MOOP', 
        'EducAttain', 'HousingStatus', 'HEAP', 'Pov_risk']

In [47]:
puma_05 = pd.read_csv("../Data/Cleaned/puma_05.csv")
puma_05.drop(['Unnamed: 0'], axis=1, inplace=True)

X5 = features(puma_05)
X5 = X5.loc[:,cols] 

train5,test5=train_test_split(X5,random_state=9,test_size=0.20)
hc5 = HillClimbSearch(train5, scoring_method=BicScore(train5))
best_model5 = hc5.estimate()

In [48]:
best_model5.edges()

[('Pov_risk', 'WorkersEquiv'),
 ('Pov_risk', 'FoodStamps'),
 ('Pov_risk', 'SchoolLunch'),
 ('Pov_risk', 'HEAP'),
 ('WorkersEquiv', 'FoodStamps'),
 ('WorkersEquiv', 'SchoolLunch'),
 ('WorkersEquiv', 'HousingStatus'),
 ('MOOP', 'Pov_risk'),
 ('MOOP', 'EducAttain'),
 ('HEAP', 'SchoolLunch'),
 ('HEAP', 'HousingStatus'),
 ('FoodStamps', 'HEAP'),
 ('EducAttain', 'WorkersEquiv'),
 ('EducAttain', 'Pov_risk')]

In [49]:
best_model5.fit(train5, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test5 = test5.astype(int)
test5.reset_index(inplace=True, drop=True)

y_pred5 = best_model5.predict(test5.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [50]:
((y_pred5.Pov_risk == test5.Pov_risk).sum()*1.0)/len(test5)

0.71943319838056685

In [51]:
metrics.confusion_matrix(test5.Pov_risk, y_pred5.Pov_risk)

array([[ 503,   73,  400],
       [ 266,  106,  493],
       [ 108,   46, 2945]])

In [52]:
metrics.precision_score(test5.Pov_risk, y_pred5.Pov_risk, average='weighted')

0.67717362904886047

In [53]:
metrics.recall_score(test5.Pov_risk, y_pred5.Pov_risk, average='weighted')

0.71943319838056685

In [94]:
print metrics.classification_report(test5.Pov_risk, y_pred5.Pov_risk,  target_names=['Poor', 'At_Risk', 'Not_Poor'])

             precision    recall  f1-score   support

       Poor       0.57      0.52      0.54       976
    At_Risk       0.47      0.12      0.19       865
   Not_Poor       0.77      0.95      0.85      3099

avg / total       0.68      0.72      0.67      4940



In [54]:
puma_06 = pd.read_csv("../Data/Cleaned/puma_06.csv")
puma_06.drop(['Unnamed: 0'], axis=1, inplace=True)

X6 = features(puma_06)
X6 = X6.loc[:,cols]
X6.dropna(inplace=True)

train6,test6=train_test_split(X6,random_state=9,test_size=0.20)
hc6 = HillClimbSearch(train6, scoring_method=BicScore(train6))
best_model6 = hc6.estimate()

In [55]:
best_model6.edges()

[('Pov_risk', 'WorkersEquiv'),
 ('Pov_risk', 'SchoolLunch'),
 ('Pov_risk', 'EducAttain'),
 ('Pov_risk', 'MOOP'),
 ('WorkersEquiv', 'SchoolLunch'),
 ('WorkersEquiv', 'EducAttain'),
 ('WorkersEquiv', 'HousingStatus'),
 ('HEAP', 'MOOP'),
 ('HEAP', 'FoodStamps'),
 ('HEAP', 'SchoolLunch'),
 ('HEAP', 'Pov_risk'),
 ('FoodStamps', 'WorkersEquiv'),
 ('FoodStamps', 'Pov_risk'),
 ('FoodStamps', 'EducAttain'),
 ('FoodStamps', 'HousingStatus')]

In [56]:
best_model6.fit(train6, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test6 = test6.astype(int)
test6.reset_index(inplace=True, drop=True)

y_pred6 = best_model6.predict(test6.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [57]:
((y_pred6.Pov_risk == test6.Pov_risk).sum()*1.0)/len(test6)

0.7053322721846399

In [58]:
metrics.confusion_matrix(test6.Pov_risk, y_pred6.Pov_risk )

array([[ 443,   78,  425],
       [ 263,  111,  552],
       [ 127,   36, 2991]])

In [59]:
metrics.precision_score(test6.Pov_risk, y_pred6.Pov_risk, average='weighted')

0.66401599436258596

In [60]:
metrics.recall_score(test6.Pov_risk, y_pred6.Pov_risk, average='weighted')

0.7053322721846399

In [93]:
print metrics.classification_report(test6.Pov_risk, y_pred6.Pov_risk,  target_names=['Poor', 'At_Risk', 'Not_Poor'])

             precision    recall  f1-score   support

       Poor       0.53      0.47      0.50       946
    At_Risk       0.49      0.12      0.19       926
   Not_Poor       0.75      0.95      0.84      3154

avg / total       0.66      0.71      0.66      5026



In [61]:
puma_07 = pd.read_csv("../Data/Cleaned/puma_07.csv")
puma_07.drop(['Unnamed: 0'], axis=1, inplace=True)

X7 = features(puma_07)
X7 = X7.loc[:,cols] 

train7,test7=train_test_split(X7,random_state=9,test_size=0.20)
hc7 = HillClimbSearch(train7, scoring_method=BicScore(train7))
best_model7 = hc7.estimate()

In [62]:
best_model7.edges()

[('Pov_risk', 'MOOP'),
 ('Pov_risk', 'SchoolLunch'),
 ('Pov_risk', 'EducAttain'),
 ('HousingStatus', 'WorkersEquiv'),
 ('HousingStatus', 'HEAP'),
 ('WorkersEquiv', 'SchoolLunch'),
 ('WorkersEquiv', 'Pov_risk'),
 ('WorkersEquiv', 'EducAttain'),
 ('HEAP', 'MOOP'),
 ('HEAP', 'FoodStamps'),
 ('HEAP', 'SchoolLunch'),
 ('HEAP', 'EducAttain'),
 ('FoodStamps', 'WorkersEquiv'),
 ('FoodStamps', 'Pov_risk')]

In [63]:
best_model7.fit(train7, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test7 = test7.astype(int)
test7.reset_index(inplace=True, drop=True)

y_pred7 = best_model7.predict(test7.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [64]:
((y_pred7.Pov_risk == test7.Pov_risk).sum()*1.0)/len(test7)

0.69006781013163143

In [92]:
print metrics.classification_report(test7.Pov_risk, y_pred7.Pov_risk,  target_names=['Poor', 'At_Risk', 'Not_Poor'])

             precision    recall  f1-score   support

       Poor       0.53      0.43      0.47       916
    At_Risk       0.48      0.11      0.18       996
   Not_Poor       0.73      0.95      0.83      3102

avg / total       0.65      0.69      0.63      5014



In [65]:
puma_08 = pd.read_csv("../Data/Cleaned/puma_08.csv")
puma_08.drop(['Unnamed: 0'], axis=1, inplace=True)

X8 = features(puma_08)
X8 = X8.loc[:,cols] 

train8,test8=train_test_split(X8,random_state=9,test_size=0.20)
hc8 = HillClimbSearch(train8, scoring_method=BicScore(train8))
best_model8 = hc8.estimate()

In [66]:
best_model8.edges()

[('Pov_risk', 'MOOP'),
 ('Pov_risk', 'FoodStamps'),
 ('Pov_risk', 'HEAP'),
 ('Pov_risk', 'EducAttain'),
 ('WorkersEquiv', 'HEAP'),
 ('WorkersEquiv', 'SchoolLunch'),
 ('WorkersEquiv', 'Pov_risk'),
 ('WorkersEquiv', 'EducAttain'),
 ('WorkersEquiv', 'HousingStatus'),
 ('SchoolLunch', 'Pov_risk'),
 ('SchoolLunch', 'HEAP'),
 ('HEAP', 'FoodStamps'),
 ('FoodStamps', 'EducAttain'),
 ('FoodStamps', 'HousingStatus'),
 ('EducAttain', 'MOOP')]

In [67]:
best_model8.fit(train8, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test8 = test8.astype(int)
test8.reset_index(inplace=True, drop=True)

y_pred8 = best_model8.predict(test8.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [68]:
((y_pred8.Pov_risk == test8.Pov_risk).sum()*1.0)/len(test8)

0.7055797388207361

In [91]:
print metrics.classification_report(test8.Pov_risk, y_pred8.Pov_risk,  target_names=['Poor', 'At_Risk', 'Not_Poor'])

             precision    recall  f1-score   support

       Poor       0.54      0.48      0.51       932
    At_Risk       0.48      0.12      0.19       945
   Not_Poor       0.75      0.95      0.84      3177

avg / total       0.66      0.71      0.66      5054



In [69]:
puma_09 = pd.read_csv("../Data/Cleaned/puma_09.csv")
puma_09.drop(['Unnamed: 0'], axis=1, inplace=True)

X9 = features(puma_09)
X9 = X9.loc[:,cols] 

train9,test9=train_test_split(X9,random_state=9,test_size=0.20)
hc9 = HillClimbSearch(train9, scoring_method=BicScore(train9))
best_model9 = hc9.estimate()

In [70]:
best_model9.edges()

[('Pov_risk', 'WorkersEquiv'),
 ('Pov_risk', 'FoodStamps'),
 ('Pov_risk', 'SchoolLunch'),
 ('Pov_risk', 'HEAP'),
 ('Pov_risk', 'MOOP'),
 ('WorkersEquiv', 'SchoolLunch'),
 ('WorkersEquiv', 'HousingStatus'),
 ('HEAP', 'FoodStamps'),
 ('FoodStamps', 'WorkersEquiv'),
 ('FoodStamps', 'SchoolLunch'),
 ('FoodStamps', 'HousingStatus'),
 ('EducAttain', 'WorkersEquiv'),
 ('EducAttain', 'Pov_risk'),
 ('EducAttain', 'HEAP'),
 ('EducAttain', 'MOOP')]

In [71]:
best_model9.fit(train9, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test9 = test9.astype(int)
test9.reset_index(inplace=True, drop=True)

y_pred9 = best_model9.predict(test9.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [72]:
((y_pred9.Pov_risk == test9.Pov_risk).sum()*1.0)/len(test9)

0.69191919191919193

In [90]:
print metrics.classification_report(test9.Pov_risk, y_pred9.Pov_risk,  target_names=['Poor', 'At_Risk', 'Not_Poor'])

             precision    recall  f1-score   support

       Poor       0.55      0.49      0.52      1025
    At_Risk       0.48      0.13      0.20      1016
   Not_Poor       0.74      0.94      0.83      3107

avg / total       0.65      0.69      0.64      5148



In [73]:
puma_10 = pd.read_csv("../Data/Cleaned/puma_10.csv")
puma_10.drop(['Unnamed: 0'], axis=1, inplace=True)

X10 = features(puma_10)
X10 = X10.loc[:,cols] 

train10,test10=train_test_split(X10,random_state=9,test_size=0.20)
hc10 = HillClimbSearch(train10, scoring_method=BicScore(train10))
best_model10 = hc10.estimate()

In [74]:
best_model10.edges()

[('Pov_risk', 'HousingStatus'),
 ('Pov_risk', 'MOOP'),
 ('Pov_risk', 'WorkersEquiv'),
 ('Pov_risk', 'SchoolLunch'),
 ('Pov_risk', 'HEAP'),
 ('Pov_risk', 'FoodStamps'),
 ('HousingStatus', 'HEAP'),
 ('WorkersEquiv', 'SchoolLunch'),
 ('WorkersEquiv', 'HousingStatus'),
 ('SchoolLunch', 'FoodStamps'),
 ('HEAP', 'FoodStamps'),
 ('HEAP', 'SchoolLunch'),
 ('EducAttain', 'WorkersEquiv'),
 ('EducAttain', 'Pov_risk'),
 ('EducAttain', 'MOOP')]

In [75]:
best_model10.fit(train10, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test10 = test10.astype(int)
test10.reset_index(inplace=True, drop=True)

y_pred10 = best_model10.predict(test10.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [76]:
((y_pred10.Pov_risk == test10.Pov_risk).sum()*1.0)/len(test10)

0.67303649914007257

In [88]:
print metrics.classification_report(test10.Pov_risk, y_pred10.Pov_risk,  target_names=['Poor', 'At_Risk', 'Not_Poor'])

             precision    recall  f1-score   support

       Poor       0.53      0.48      0.50      1050
    At_Risk       0.46      0.23      0.31      1126
   Not_Poor       0.74      0.90      0.81      3057

avg / total       0.64      0.67      0.64      5233



In [77]:
puma_11 = pd.read_csv("../Data/Cleaned/puma_11.csv")
puma_11.drop(['Unnamed: 0'], axis=1, inplace=True)

X11 = features(puma_11)
X11 = X11.loc[:,cols] 

train11,test11=train_test_split(X11,random_state=9,test_size=0.20)
hc11 = HillClimbSearch(train11, scoring_method=BicScore(train11))
best_model11 = hc11.estimate()

In [78]:
best_model11.edges()

[('Pov_risk', 'WorkersEquiv'),
 ('Pov_risk', 'FoodStamps'),
 ('Pov_risk', 'SchoolLunch'),
 ('Pov_risk', 'HEAP'),
 ('Pov_risk', 'MOOP'),
 ('WorkersEquiv', 'SchoolLunch'),
 ('WorkersEquiv', 'HousingStatus'),
 ('SchoolLunch', 'HEAP'),
 ('HEAP', 'HousingStatus'),
 ('FoodStamps', 'WorkersEquiv'),
 ('FoodStamps', 'SchoolLunch'),
 ('FoodStamps', 'HEAP'),
 ('EducAttain', 'WorkersEquiv'),
 ('EducAttain', 'FoodStamps'),
 ('EducAttain', 'Pov_risk'),
 ('EducAttain', 'MOOP')]

In [79]:
best_model11.fit(train11, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test11 = test11.astype(int)
test11.reset_index(inplace=True, drop=True)

y_pred11 = best_model11.predict(test11.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [80]:
((y_pred11.Pov_risk == test11.Pov_risk).sum()*1.0)/len(test11)

0.65649001536098306

In [87]:
print metrics.classification_report( test11.Pov_risk,y_pred11.Pov_risk, target_names=['Poor', 'At_Risk', 'Not_Poor'])

             precision    recall  f1-score   support

       Poor       0.49      0.49      0.49      1040
    At_Risk       0.41      0.11      0.18      1121
   Not_Poor       0.72      0.91      0.81      3047

avg / total       0.61      0.66      0.61      5208



In [ ]:
puma_12 = pd.read_csv("../Data/Cleaned/puma_12.csv")
puma_12.drop(['Unnamed: 0'], axis=1, inplace=True)

X12 = features(puma_12)
X12 = X12.loc[:,cols] 

train12,test12=train_test_split(X12,random_state=9,test_size=0.20)
hc12 = HillClimbSearch(train12, scoring_method=BicScore(train12))
best_model12 = hc12.estimate()

In [ ]:
best_model12.edges()

In [ ]:
best_model12.fit(train12, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test12 = test12.astype(int)
test12.reset_index(inplace=True, drop=True)

y_pred12 = best_model12.predict(test12.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [ ]:
((y_pred12.Pov_risk == test12.Pov_risk).sum()*1.0)/len(test12)

In [ ]:
puma_13 = pd.read_csv("../Data/Cleaned/puma_13.csv")
puma_13.drop(['Unnamed: 0'], axis=1, inplace=True)

X13 = features(puma_13)
X13 = X13.loc[:,cols] 

train13,test13=train_test_split(X13,random_state=9,test_size=0.20)
hc13 = HillClimbSearch(train13, scoring_method=BicScore(train13))
best_model13 = hc13.estimate()

In [ ]:
best_model13.edges()

In [ ]:
best_model13.fit(train13, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test13 = test13.astype(int)
test13.reset_index(inplace=True, drop=True)

y_pred13 = best_model13.predict(test13.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [ ]:
((y_pred13.Pov_risk == test13.Pov_risk).sum()*1.0)/len(test13)

In [ ]:
puma_14 = pd.read_csv("../Data/Cleaned/puma_14.csv")
puma_14.drop(['Unnamed: 0'], axis=1, inplace=True)

X14 = features(puma_14)
X14 = X14.loc[:,cols] 

train14,test14=train_test_split(X14,random_state=9,test_size=0.20)
hc14 = HillClimbSearch(train14, scoring_method=BicScore(train14))
best_model14 = hc14.estimate()

In [ ]:
best_model14.edges()

In [ ]:
best_model14.fit(train14, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
test14 = test14.astype(int)
test14.reset_index(inplace=True, drop=True)

y_pred14 = best_model14.predict(test14.loc[:,[u'HEAP', u'HousingStatus', u'WorkersEquiv', u'SchoolLunch', u'MOOP',
       u'EducAttain', u'FoodStamps']])

In [ ]:
((y_pred14.Pov_risk == test14.Pov_risk).sum()*1.0)/len(test14)